<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E3%82%92%E6%A4%9C%E8%A8%8E%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
!mkdir input output

In [4]:
!kaggle competitions download -c foursquare-location-matching
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 92% 146M/159M [00:00<00:00, 173MB/s]
100% 159M/159M [00:00<00:00, 173MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [7]:
import pandas as pd
import numpy as np
import math
import os

## データ読み込み

In [20]:
%cd /content/output
data_root = '../input/foursquare-location-matching'
df = pd.read_csv(os.path.join(data_root, 'train.csv'))

/content/output


### カテゴリの出現回数をDICTに格納

In [21]:
cat_count = dict(df.groupby('categories')['categories'].count().sort_values(ascending=False))

## poiでペアを作成

In [22]:
pat = df.groupby('point_of_interest')['categories'].apply(list).apply(set)
# cats2idx = {x : i for i, x in enumerate(list(df['categories'].unique()))}


In [41]:
from collections import defaultdict
import math 

pairs = defaultdict(int)
for e in pat : 
  if len(e) == 1 : continue
  x = list(e)
  for i in range(len(x)) :
    for j in range(i+1, len(x)) :
      if type(x[i]) == float and math.isnan(x[i]) : continue
      if type(x[j]) == float and math.isnan(x[j]) : continue
      pairs[(x[i], x[j])] += 1

pairs = [(pairs[x], x[0], x[1]) for x in pairs]
pairs.sort(reverse=True)

## 別名のマージ

In [69]:
cdict = defaultdict(str)
cnt = 0
for n, x, y in pairs :
  if n < 100: break
  nx, ny = cat_count[x], cat_count[y]
  # print(n, x,"-", y, nx, ny)
  if nx < ny : 
    nx, ny = ny, nx
    x,y = y,x
  if y not in cdict :
    if x in cdict :
      x = cdict[x]
    cdict[y] = x
  # print(n, x,"-", y, nx, ny)
  # if cnt == 10: break
  cnt+=1

## categoriesに反映

In [70]:
def f(x) :
  if type(x)==float and math.isnan(x) : return x
  if x in cdict : return cdict[x]
  return x

df['categories2'] = df['categories'].apply(f)

In [74]:
df.sample(20)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest,categories2
578068,E_81f4a7afa9c2e9,SUBWAY,44.918155,-93.502702,4711 County Road 101,Minnetonka,MN,55345,US,https://restaurants.subway.com/united-states/m...,+19529354676,"Sandwich Places, Fast Food Restaurants",P_95bb9feedcd525,Sandwich Places
609246,E_88f5bb08f503ae,"Leaps And Bounds Pt And Ot, Pllc",40.772014,-73.925702,21-14 Newtown Ave,Astoria,NY,11102,US,http://leapsandboundspeds.com,7185450958,NaN,P_f9f6b097c8e782,NaN
588824,E_84589f7f176ce8,Coup de Coeur,48.884544,2.321564,Rue des Batignolles,Paris,Île-de-France,75017,FR,NaN,NaN,Furniture / Home Stores,P_e5ff9d7eda2f8c,Furniture / Home Stores
461278,E_67a12e15479b10,Mont Bleu Hotel & Spa,38.963914,-119.939117,55 Highway 50,Stateline,NV,89449,US,http://www.montbleuresort.com,8888297630,"Hotels, Spas, Resorts",P_6d59dc89c7095a,"Hotels, Spas, Resorts"
792925,E_b218c7cb415204,Site matur,-0.278760,100.283930,NaN,Sungai Puar,Sumatera Barat,NaN,ID,NaN,NaN,NaN,P_66615b51447b4e,NaN
833740,E_bb3bc309a172ee,สนามบาสเก็ตบอลธิดานุเคราะห์,7.011699,100.473958,NaN,Hat Yai,Songkhla,NaN,TH,NaN,NaN,College Basketball Courts,P_fc62e697239cce,College Basketball Courts
81067,E_12207eb815a37f,Freya Baby & Kids Shop,-6.718337,108.570577,Jl. Pasuketan No. 1,Cirebon,NaN,NaN,ID,NaN,NaN,"Kids Stores, Baby Stores",P_a20460c1ef7c78,"Kids Stores, Baby Stores"
1089926,E_f4edd0776c47b2,大阪屋台居酒屋 満マル 高田馬場店,35.714220,139.703770,高田馬場3-1-4,新宿区,東京都,169-0075,JP,NaN,+81359371750,Sake Bars,P_51048e0df92064,Sake Bars
1022238,E_e5af3d0680ee61,TB 310,41.083808,29.051519,Boğaziçi Üniversitesi Güney Kampüsü,İstanbul,Türkiye,NaN,TR,NaN,NaN,College Classrooms,P_d00ec2a4b57f9a,College Classrooms
7382,E_01aef05ef1ce8e,40 лет победы 5,55.816778,37.971247,NaN,NaN,NaN,NaN,RU,NaN,NaN,Residential Buildings (Apartments / Condos),P_8582ba60c16efe,Residential Buildings (Apartments / Condos)


In [72]:
cdict

defaultdict(str,
            {'Airport Services': 'Airports',
             'Airport Terminals': 'Airports',
             'American Restaurants': 'Fast Food Restaurants',
             'Asian Restaurants': 'Chinese Restaurants',
             'Auto Dealerships': 'Automotive Shops',
             'Auto Dealerships, Automotive Shops': 'Auto Dealerships',
             'Auto Workshops, Automotive Shops, Auto Garages': 'Automotive Shops',
             'Automotive Shops, Auto Dealerships': 'Automotive Shops',
             'Automotive Shops, Auto Workshops, Auto Garages': 'Automotive Shops',
             'Banks, ATMs': 'Banks',
             'Banks, Buildings': 'Banks',
             'Banks, Offices': 'Banks',
             'Beaches, Surf Spots': 'Beaches',
             'Bed & Breakfasts': 'Hotels',
             'Boutiques': 'Clothing Stores',
             'Buddhist Temples': 'Temples',
             'Buildings': 'Offices',
             'Buildings, Offices': 'Offices',
             'Buildings, Reside